# Update Certificates  
SSLCertVerificationError


In [9]:
! pip3 install certifi


In [23]:
import os
import certifi


os.environ['SSL_CERT_FILE'] = certifi.where()



# Downlaoding the Dataset 

In [12]:
import convokit

In [11]:
from convokit import Corpus, download

In [1]:

corpus = Corpus(filename=download("movie-corpus"))


Load data based on where it was downloaded by previous cell

In [9]:
corpus = Corpus(filename="/Users/wajeeh/.convokit/downloads/movie-corpus")

## Perform NER with SpaCy for Forced Directed Graph

In [8]:
import spacy
import json

In [33]:
def generate_entity_network():
        nlp = spacy.load("en_core_web_sm")
        co_occurrence = {}

        # for utterance in list(corpus.iter_utterances())[:10000]:
        for utterance in list(corpus.iter_utterances()):
            doc = nlp(utterance.text)
            entities = [ent.text for ent in doc.ents if ent.label_ in ['PERSON', 'ORG', 'GPE']]
            for i, entity in enumerate(entities):
                for adjacent in entities[i + 1:]:
                    if entity != adjacent:
                        pair = tuple(sorted([entity, adjacent]))
                        co_occurrence[pair] = co_occurrence.get(pair, 0) + 1

        nodes = list({entity for pair in co_occurrence for entity in pair})
        links = [{"source": pair[0], "target": pair[1], "value": count} for pair, count in co_occurrence.items()]

        # Save to JSON
        with open('entity_network.json', 'w') as f:
            json.dump({"nodes": [{"id": node} for node in nodes], "links": links}, f)

generate_entity_network()

## LDA Topic Modelling for Bubble Charts 

preprocess text data


In [19]:
! /usr/bin/python -m certifi


/System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: No module named certifi


In [28]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

os.environ['SSL_CERT_FILE'] = certifi.where()
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)


True

In [25]:
stop_words = set(stopwords.words('english'))

for utt in corpus.iter_utterances():
    text = utt.text.lower()  # Convert to lowercase
    tokens = word_tokenize(text)  # Tokenize text
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]  # Remove stopwords
    utt.text = " ".join(filtered_tokens)  # Rejoin tokens into a clean text

perform LDA topic modelling

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

# Create a document-term matrix using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(utt.text for utt in corpus.iter_utterances())

# Perform LDA topic modeling
num_topics = 30  # Adjust the number of topics as needed
lda = LatentDirichletAllocation(n_components=num_topics, random_state=0)
lda.fit(X)

# Extract topics, top words, and topic weights
topic_keywords = []
topic_weights = np.sum(lda.transform(X), axis=0)  # Sum of topic distributions

for topic_idx, topic in enumerate(lda.components_):
    top_words_idx = topic.argsort()[::-1][:10]  # Get the top 10 words for each topic
    top_words = [vectorizer.get_feature_names_out()[i] for i in top_words_idx]
    topic_weight = topic_weights[topic_idx]  # Weight of the topic
    topic_keywords.append({
        "topic_id": topic_idx,
        "keywords": top_words,
        "value": topic_weight  # Add the topic weight here
    })

# Convert topic data to JSON
import json

data_for_d3 = {
    "nodes": topic_keywords  # Use 'nodes' to align with your D3.js code
}

with open("topics_data.json", "w") as json_file:
    json.dump(data_for_d3, json_file)


# NER for Radial Network Graph

In [32]:

import spacy
import json
from collections import defaultdict

def generate_radial_network():
    nlp = spacy.load("en_core_web_sm")
    co_occurrence = defaultdict(int)
    entity_counts = defaultdict(int)
    entity_types = {}

    # Process utterances to count co-occurrences and entity frequencies
    # for utterance in list(corpus.iter_utterances())[:10000]:
    for utterance in list(corpus.iter_utterances()):
        doc = nlp(utterance.text)
        entities = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ in ['PERSON', 'ORG', 'GPE']]
        for i, (entity, type) in enumerate(entities):
            entity_counts[entity] += 1
            entity_types[entity] = type
            for adjacent, _ in entities[i + 1:]:
                if entity != adjacent:
                    pair = tuple(sorted([entity, adjacent]))
                    co_occurrence[pair] += 1

    # Prepare nodes and links for the radial network graph
    nodes = [{"id": entity, "type": entity_types[entity], "weight": entity_counts[entity]} for entity in entity_counts]
    links = [{"source": pair[0], "target": pair[1], "value": count} for pair, count in co_occurrence.items()]

    # Save to a new JSON file
    with open('radial_network_data.json', 'w') as f:
        json.dump({"nodes": nodes, "links": links}, f)

generate_radial_network()
